# Jet in cross-flow

Andreas Kitzmüller, Simon Mader

## 1. Pre-considerations and preparations

Many different turbulence solvers, hard to judge which one is realistic.

Hence: never trust stand-alone CFD simulations

Problem: we have no experimental data to compare with

Solution: ensure laminar flow and pick a laminar solver, i.e. icoFoam.

### 1.1 Ensuring laminar flow

First, we define the kinematic viscosity of air, as used in the simulation, as a global variable:

In [3]:
NU_AIR = 1.5e-5



 find the right dimension to ensure laminar flow. The Reynolds $\text{Re}$ number is given by:

 $\text{Re} = \frac{u \cdot D_h}{\nu_{air}}$, with $D_h = \frac{A}{P}$, with area $A=s$ and perimeter $4 \cdot s$. $\Rightarrow D_h = s \Rightarrow \text{Re} = \frac{u \cdot s}{\nu_{air}}$

In [5]:
Reynolds = lambda s=1.0, u=1.0 : u * s / NU_AIR

In order to find a good geometry, we solve the above formula for u, to be able to set certain side length and velocities and check their Reynolds numbers:

$u = \frac{\text{Re} \cdot \nu_{air}}{s}$

In [6]:
find_velocity = lambda s=1.0, Re=2000.0: Re * NU_AIR / s

To be on the safe side and allow for reasonable cases, we aim for Re less or equal than 2000 for the main pipe and less or equal than 200 for the small duct. In addition, we want to allow for velocities of up to $1\,\mathtt{m/s}$. Therefore, we choose:

In [7]:
print("maximal initial velocity small duct:", find_velocity(s=0.00375, Re=200), "m/s")
print("maximal initial velocity main pipe:", find_velocity(s=0.01125, Re=2000), "m/s")

maximal initial velocity small duct: 0.8 m/s
maximal initial velocity main pipe: 2.666666666666667 m/s


Check the Reynolds number with the initial conditions for both the main pipe and the duct:

In [8]:
print(Reynolds(s=0.00375, u=0.6))
print(Reynolds(s=0.01125, u=0.6))

149.99999999999997
450.0


The laminar and incompressible solver we choose is icoFoam, which solves the following equations using the PISO algorithm:

1. continuity equation: $\nabla \cdot \mathbf{u} = 0$

2. momentum equation: $\frac{\partial \mathbf{u}}{\partial t} + \nabla \cdot (\mathbf{u} \otimes \mathbf{u}) - \nabla \cdot (\nu \nabla \mathbf{u}) = -\nabla p$

$\mathbf{u}$ ... velocity field, $\nabla p$ ... kinematic pressure

We modify this solver to include mixing of the 2 fluids.

<img src="CFL_Pictures/CFL2.png" alt="description" style="float: right; margin-left: 10px; width: 500px;">
<img src="CFL_Pictures/CFL.png" alt="description" style="float: right; margin-left: 10px; width: 365.5px;">

**Courant–Friedrichs–Lewy (CFL) Number**

The CFL number describes the relationship between:

- the flow velocity $u$,
- the grid resolution $Δx$,
- and the numerical time stepwidth $Δt$.
- For a $n$-dimensional problem: $\text{CFL} = \sum_{i=1}^n\frac{u_i\,\Delta t}{\Delta x_i} \leq C_{max} \approx 1$

The CFL number tells us **how far information travels during one time step,
measured in units of cell sizes**.

- Solver Differential scheme: upwind -> only considers 1 neighboring cell upstream to solve Diff. eq.

- Interpretation:
Solver can only use information from neighboring cells. If physical signal moves faster than one cell per time step (CFL > 1) -> information “tunnels through” the neighbor cell without ever being seen by the solver

SOURCE: https://www.openfoam.com/documentation/guides/latest/doc/guide-fos-field-CourantNo.html

### 1.3 Check plausibility of the solution

$\textbf{Continuity Error Types in OpenFOAM}$

OpenFOAM computes three related measures of continuity error for a flux field `φ`.  
These errors quantify how well the discrete mass-conservation equation is satisfied: $\nabla \phi = 0$ (for face flux field $\phi$)

<table>
<tr>
<td style="vertical-align: top; width: 50%; padding-right: 20px;">
<strong>Local continuity error:</strong><br>

$
\epsilon_{\text{local}} = \Delta t \left\langle \; |x| \; \right\rangle
$

Meaning:  
- This error shows **how badly continuity is violated inside cells**, on average.
- $x$ is the divergence of the face flux field ($\nabla \phi = x \neq 0$).  
- The absolute value ensures that positive and negative divergence do not cancel out.  

</td>
<td style="vertical-align: top; width: 50%;">
<strong>Global continuity error:</strong><br>

$
\epsilon^t_{\text{global}} = \Delta t \left\langle \; x \; \right\rangle
$

Meaning:
- Measures the *net* continuity imbalance over the entire domain.  
- Positive and negative contributions of individual cells can cancel out
- This error detects **global mass gain or mass loss** in each timestep.

<br>

<strong>Cumulative continuity error:</strong><br>

$
\epsilon_{\text{cum}} = \sum_t \epsilon^t_{\text{global}}
$

Meaning:
- A running sum of the **global** error over all timesteps.  
- Indicates whether the simulation accumulates mass over time.  
- Useful for checking long-term conservation stability.

</td>
</tr>
</table>

SOURCE: https://doc.openfoam.com/2312/tools/post-processing/function-objects/field/continuityError/


### 1.4 Computational domain

The main pipe should be longer in the direction of the main flow to ensure uniformity at the end of the domain. This way, the correctness of the solution should be guaranteed.

## 2. Simulations and post-processing

### Case 1: $u_{pipe} = u_{duct} = 0.1 m/s$

Simulate for 3 seconds to get a steady result (based on cA and cB).

#### Courant-Friedrichs-Lewy (CFL) number

<div style="display: flex; gap: 20px;">
  <video width="850" controls>
    <source src="U1_0_1_U2_0_1/videos/Co_smoothed.mp4" type="video/mp4">
  </video>

  <img src="U1_0_1_U2_0_1/plots/Courant_number_plot.png" width="850">
</div>

### concentrations of streams A (cA) and B (cB)

<div style="display: flex; gap: 20px;">
  <video width="850" controls>
    <source src="U1_0_1_U2_0_1/videos/cA_smoothed.mp4" type="video/mp4">
  </video>

  <video width="850" controls>
    <source src="U1_0_1_U2_0_1/videos/cB_smoothed.mp4" type="video/mp4">
  </video>
</div>

### velocity and pressure fields

<div style="display: flex; gap: 20px;">
  <video width="850" controls>
    <source src="U1_0_1_U2_0_1/videos/U_smoothed.mp4" type="video/mp4">
  </video>

  <video width="850" controls>
    <source src="U1_0_1_U2_0_1/videos/p_smoothed.mp4" type="video/mp4">
  </video>
</div>

### isosurfaces

<div style="display: flex; gap: 20px;">
  <video width="850" controls>
    <source src="U1_0_1_U2_0_1/videos/U1_0_1_U2_0_1_IsoSurface_cA_0_5.ogv.mp4" type="video/mp4">
  </video>

  <video width="850" controls>
    <source src="U1_0_1_U2_0_1/videos/U1_0_1_U2_0_1_IsoSurface_cB_0_5.ogv.mp4" type="video/mp4">
  </video>
</div>

### Computation stats

<div style="display: flex; gap: 20px;">
  <img controls src="U1_0_1_U2_0_1/plots/continuityError_plot.png" width="850" />

  <img controls src="U1_0_1_U2_0_1/plots/computation_time_plot.png" width="850" />
</div>

### Case 2: $u_{pipe} = 0.1 m/s,  = u_{duct} = 0.6 m/s$

Simulate for 1.25 seconds to get a steady result (based on cA and cB).

#### Courant-Friedrichs-Lewy (CFL) number

<div style="display: flex; gap: 20px;">
  <video width="850" controls>
    <source src="U1_0_1_U2_0_6/videos/Co_smoothed.mp4" type="video/mp4">
  </video>

  <img controls src="U1_0_1_U2_0_6/plots/Courant_number_plot.png" width="850" />
</div>

### concentrations of streams A (cA) and B (cB)

<div style="display: flex; gap: 20px;">
  <video width="850" controls>
    <source src="U1_0_1_U2_0_6/videos/cA_smoothed.mp4" type="video/mp4">
  </video>

  <video width="850" controls>
    <source src="U1_0_1_U2_0_6/videos/cB_smoothed.mp4" type="video/mp4">
  </video>
</div>

### velocity and pressure fields

<div style="display: flex; gap: 20px;">
  <video width="850" controls>
    <source src="U1_0_1_U2_0_6/videos/U_smoothed.mp4" type="video/mp4">
  </video>

  <video width="850" controls>
    <source src="U1_0_1_U2_0_6/videos/p_smoothed.mp4" type="video/mp4">
  </video>
</div>

### isosurfaces

<div style="display: flex; gap: 20px;">
  <video width="850" controls>
    <source src="U1_0_1_U2_0_6/videos/U1_0_1_U2_0_6_IsoSurface_cA_0_5.mp4" type="video/mp4">
  </video>

  <video width="850" controls>
    <source src="U1_0_1_U2_0_6/videos/U1_0_1_U2_0_6_IsoSurface_cB_0_5.mp4" type="video/mp4">
  </video>
</div>

### Computation stats

<div style="display: flex; gap: 20px;">
  <img controls src="U1_0_1_U2_0_6/plots/continuityError_plot.png" width="850" />

  <img controls src="U1_0_1_U2_0_6/plots/computation_time_plot.png" width="850" />
</div>

### Case 3: $u_{pipe} = 0.6 m/s,  = u_{duct} = 0.1 m/s$

Simulate for 0.75 seconds to get a steady result (based on cA and cB).

#### Courant-Friedrichs-Lewy (CFL) number

<div style="display: flex; gap: 20px;">
  <video width="850" controls>
    <source src="U1_0_6_U2_0_1/videos/Co_smoothed.mp4" type="video/mp4">
  </video>

  <img controls src="U1_0_6_U2_0_1/plots/Courant_number_plot.png" width="850" />
</div>

### concentrations of streams A (cA) and B (cB)

<div style="display: flex; gap: 20px;">
  <video width="850" controls>
    <source src="U1_0_6_U2_0_1/videos/cA_smoothed.mp4" type="video/mp4">
  </video>

  <video width="850" controls>
    <source src="U1_0_6_U2_0_1/videos/cB_smoothed.mp4" type="video/mp4">
  </video>
</div>

### velocity and pressure fields

<div style="display: flex; gap: 20px;">
  <video width="850" controls>
    <source src="U1_0_6_U2_0_1/videos/U_smoothed.mp4" type="video/mp4">
  </video>

  <video width="850" controls>
    <source src="U1_0_6_U2_0_1/videos/p_smoothed.mp4" type="video/mp4">
  </video>
</div>

### isosurfaces

<div style="display: flex; gap: 20px;">
  <video width="850" controls>
    <source src="U1_0_6_U2_0_1/videos/U1_0_6_U2_0_1_IsoSurface_cA_0_5.mp4" type="video/mp4">
  </video>

  <video width="850" controls>
    <source src="U1_0_6_U2_0_1/videos/U1_0_6_U2_0_1_IsoSurface_cB_0_5.mp4" type="video/mp4">
  </video>
</div>

### Computation stats

<div style="display: flex; gap: 20px;">
  <img controls src="U1_0_6_U2_0_1/plots/continuityError_plot.png" width="850" />

  <img controls src="U1_0_6_U2_0_1/plots/computation_time_plot.png" width="850" />
</div>